In [ ]:
# Dictionaary for retrieving the average sell price multiplier per framing level for unfertilized soil
normalAvgPrices = {
    0: 1.01,
    1: 1.03,
    2: 1.05,
    3: 1.07,
    4: 1.09,
    5: 1.10,
    6: 1.12,
    7: 1.14,
    8: 1.16,
    9: 1.17,
    10: 1.19,
    11: 1.20,
    12: 1.22,
    13: 1.23,
    14: 1.25
}

# Dictionary for retrieving the average sell price multiplier per forming level for soil fertilized with Basic Fertilizer
basicAvgPrices = {
    0: 1.04,
    1: 1.08,
    2: 1.11,
    3: 1.14,
    4: 1.17,
    5: 1.20,
    6: 1.23,
    7: 1.25,
    8: 1.28,
    9: 1.30,
    10: 1.32,
    11: 1.33,
    12: 1.34,
    13: 1.35,
    14: 1.36
}

# Dictionary for retrieving the average sell price multiplier per forming level for soil fertilized with Quality Fertilizer
qualityAvgPrices = {
    0: 1.07,
    1: 1.12,
    2: 1.17,
    3: 1.21,
    4: 1.25,
    5: 1.28,
    6: 1.31,
    7: 1.33,
    8: 1.34,
    9: 1.36,
    10: 1.38,
    11: 1.39,
    12: 1.41,
    13: 1.43,
    14: 1.44
}

# Dictionary for retrieving the average sell price multiplier per forming level for soil fertilized with Deluxe Fertilizer
deluxeAvgPrices = {
    0: 1.32,
    1: 1.36,
    2: 1.40,
    3: 1.44,
    4: 1.47,
    5: 1.51,
    6: 1.55,
    7: 1.58,
    8: 1.61,
    9: 1.64,
    10: 1.67,
    11: 1.70,
    12: 1.73,
    13: 1.76,
    14: 1.77
}

# TODO:

## Functionality:

### Important:
    * Implement a widget for the selection of the Tiller and/or Agriculturalist professions
        - Modify the process_growth_fertil() function to handle the presence of the Agriculturalist profession
        - Modify the process_sell_mult function to handle the presence of the Tiller profession
    * Implement dependency between the fertilizer toggle and the graph
        - Modify the update_graph() function to graph fertilizer information only when toggled

### If Time Allows:
    * Allow for sorting by various fields (Season, Mature Time, Type of Crop)

## Data Visualization:
    * Implement proper color strokes for bars, with consideration for accessibility
        - Involves implementation of a corresponding legend, which should be modified depending on whether fertilizers are being graphed
    * Polish the spatial dimensions of the graph (padding between bars and groups, adaptive y-axis bounds, rotation of text)

In [ ]:
from functools import partial
from IPython.display import display
import bqplot
import ipywidgets as widgets
import math
import pandas as pd
import traitlets

dfBase = pd.read_csv('Stardew_Valley_Crops.csv')

dfBase['Gold/Day'] = {}
dfBase['XP/Day'] = {}

dfBase

# Functions

In [ ]:
# Function for calculating the Gold/XP per Day averages under the current criteria
# Parameters:
#             df - The independently maintained dataframe to modify 
#       sellMult - The multiplier to apply to the base sell price of each crop
#     matureMult - The multiplier to apply to the base maturity time of each crop

def calc_crop_avgs(df, sellMult, matureMult):
    for i in range (0,43):
        # In-place modification of the current crop's base maturation time
        df.at[i, 'Mature Time (Days)'] = max(math.floor(dfBase.at[i, 'Mature Time (Days)'] * matureMult), 1)
        
        entry = df.iloc[i]
        seasons = entry['Season'].split(sep=",")
        daysGrowing = matureTime = entry['Mature Time (Days)'] 
        regrowTime = entry['Regrowth Time (Days)']
        harvestYield = entry['Yield (per Harvest)']
        cost = entry['Seed Cost (Gold)']
        sellPrice = entry['Sell Price (Gold)']
        xpGain = entry['XP']
        maxHarvests = 1

        # More than 1 harvest if crop regrows
        if not pd.isna(regrowTime):
            # 28 days to a season, subtract 1 day because final day (night) marks the turn to the next season
            maxDays = len(seasons) * 28 - 1
        
            # Divide remaining days (after first harvest) by the regrowth time to calculate number of additional harvests
            maxHarvests = 1 + math.floor((maxDays - matureTime) / regrowTime)
            # print("Max harvests: " + str(maxHarvests))
       
            # Calculate the total number of days spent growing (ONLY which result in a harvest); season may change in the middle of regrowth
            daysGrowing += (maxHarvests - 1) * regrowTime

        # Only the first in a yield may be of a higher quality, the rest of the yield will be of normal quality (base sell price applioed)
        yieldPrice = sellPrice * (sellMult + harvestYield - 1)

        # Averaging the relevant returns of the current crop over the days (which result in a harvest) spent growing
        goldPerDay = ((maxHarvests * yieldPrice) - cost) / daysGrowing
        xpPerDay = maxHarvests * xpGain / daysGrowing

        df.at[i, 'Gold/Day'] = goldPerDay
        df.at[i, 'XP/Day'] = xpPerDay


# Callback function which toggles the visualization of fertilizers
# Parameters:
#         change - The checkbox widget which a change has been observed in

def toggle_dropdowns(change):
    checked = change['new']
    growthFertilDrop.disabled = qualFertilDrop.disabled = not checked
    
    if checked:
        with out:
            display(fertils)
    else:
        # Reset the fertilizer dropdown widgets to their default states and remove them from the current display
        growthFertilDrop.value = 1
        qualFertilDrop.value = ''
        out.clear_output()

    update_graph()


# Callback function which processes any observed changes made which affect the sell price multiplier
# Parameters:
#         change - The widget in which a relevant change has been observed in

def process_sell_mult(change):
    # Current farming level initialized by referencing the value; the BoundedIntText widget might not be the one which has been modified
    currLevel = farmingLevelField.value
    sellMult = 0.0

    match qualFertilDrop.value:
        case '':
            sellMult = normalAvgPrices[currLevel]
        case 'Basic Fertilizer':
            sellMult = basicAvgPrices[currLevel]
        case 'Quality Fertilizer':
            sellMult = qualityAvgPrices[currLevel]
        case 'Deluxe Fertilizer':
            sellMult = deluxeAvgPrices[currLevel]
        case _:
            print("Something went wrong while processing the farming level")

    # A change in farming level requires recalculation for each maintained dataframe, then update the graph
    calc_crop_avgs(dfBase, normalAvgPrices[currLevel], 1)
    calc_crop_avgs(dfQual, sellMult, 1)
    calc_crop_avgs(dfGrowth, normalAvgPrices[currLevel], growthFertilDrop.value)
    update_graph()


# Callback function which processes any observed changes to the selected growth fertilizer
# Parameters:
#         change - The widget in which a relevant change has been observed in

def process_growth_fertil(change):
    matureMult = growthFertilDrop.value
    calc_crop_avgs(dfGrowth, normalAvgPrices[farmingLevelField.value], matureMult)
    update_graph()

# Callback function which modifies the current 5 crops being displayed within the graph
# Parameters:
#         button - The button which was clicked

def modify_selection(xBounds, button):
    if button.tooltip == "prev":
        xBounds[0] = max(0, xBounds[0] - 4)
        xBounds[1] = xBounds[0] + 4
    else:
        xBounds[1] = min(xBounds[1] + 4, 42)
        xBounds[0] = xBounds[1] - 4

    update_graph()

# Function used which initializes the default state of the graph
# Returns a bqplot Figure
    
def init_graph(): 
    out.clear_output()
    
    # Scales
    x_scl = bqplot.OrdinalScale()
    y_scl = bqplot.LinearScale(min=0, max=60)
    
    # Mark
    bars = bqplot.Bars(x=dfBase.loc[xBounds[0]:xBounds[1], 'Crop'].tolist(),
                       y=[dfBase.loc[xBounds[0]:xBounds[1], 'Gold/Day'].tolist(),
                          dfQual.loc[xBounds[0]:xBounds[1], 'Gold/Day'].tolist(),
                          dfGrowth.loc[xBounds[0]:xBounds[1], 'Gold/Day'].tolist()],
                       scales={'x': x_scl, 'y': y_scl},
                       tooltip=bqplot.Tooltip(fields=['y']),
                       color_mode='element',
                       colors=['#ff9900', '#ff5050', '#9933ff'],
                       type='grouped',
                       padding=0.25,
                       labels=["No Fertilizer", "Crop-quality Fertilizer", "Growth Fertilizer"],
                       display_legend=True)

    # Axes
    x_ax = bqplot.Axis(scale=x_scl, label="Crop", grid_lines='none')
    y_ax = bqplot.Axis(scale= y_scl, label="Gold/Day", orientation='vertical')

    # Figure
    fig = bqplot.Figure(marks=[bars], axes=[x_ax, y_ax], title="Stardew Valley Crop Efficiency", legend_location='top-right')

    return (fig, y_scl)


# Function which updates the current state of the visualization, depending on currently selected widget values.
#    Placeholder parameter to allow for updating of graphed column

def update_graph(placeholder=None):
    # Determine y-axis bounds of base bar values
    yMin = min(0, dfBase.loc[xBounds[0]:xBounds[1]][graphedValueSel.value].min())
    yMax = dfBase.loc[xBounds[0]:xBounds[1]][graphedValueSel.value].max()

        
    with fig.hold_sync():
        fig.marks[0].x = dfBase.loc[xBounds[0]:xBounds[1], 'Crop'].tolist()
        fig.marks[0].y = dfBase.loc[xBounds[0]:xBounds[1], graphedValueSel.value].tolist()

        # If toggled, determine y-axis bounds from and graph all dataframes
        if fertilToggle.value:
            fig.marks[0].y = [
                                dfBase.loc[xBounds[0]:xBounds[1], graphedValueSel.value].tolist(),
                                dfQual.loc[xBounds[0]:xBounds[1], graphedValueSel.value].tolist(),
                                dfGrowth.loc[xBounds[0]:xBounds[1], graphedValueSel.value].tolist()
                             ]
            
            yMin = min(
                        yMin,
                        dfQual.loc[xBounds[0]:xBounds[1]][graphedValueSel.value].min(),
                        dfGrowth.loc[xBounds[0]:xBounds[1]][graphedValueSel.value].min()
                      )
        
            yMax = max(
                        yMax,
                        dfQual.loc[xBounds[0]:xBounds[1]][graphedValueSel.value].max(),
                        dfGrowth.loc[xBounds[0]:xBounds[1]][graphedValueSel.value].max()
                      )
            
        y_scl.min = round_down(yMin)
        y_scl.max = round_up(yMax)
        fig.axes[1].label = graphedValueSel.value


# Function for rounding the y-axis upper-bound to the next highest 10
# Parameters:
#              x - The largest value currently graphed

def round_up(x):
    return int(math.ceil(x / 10.0) * 10.0)


# Function for rounding the y-axis lower-bound to the next lowest 10
# Parameters:
#              x - The smallest value currently graphed

def round_down(x):
    return int(math.floor(x / 10.0) * 10.0)
    

# Widgets

In [ ]:
# Interactive widget for graphing the previous 5 crops
prevPageButton = widgets.Button(
    description="<",
    tooltip="prev",
    layout=widgets.Layout(width='auto'),
)

# Interactive widget for graphing the next 5 crops
nextPageButton = widgets.Button(
    description=">",
    tooltip="next",
    layout=widgets.Layout(width='auto')
)

# Interactive widget for selecting whether to graph for Gold/Day or XP/Day columns
graphedValueSel = widgets.Select(
    options=['Gold/Day', 'XP/Day'],
    value='Gold/Day',
    description="Field to graph:",
    rows=2,
    style={'description_width': 'initial'},
)

# Interactive widget for specifying the farming level to reference for the graph
farmingLevelField= widgets.BoundedIntText(
    value=0,
    min=0,
    max=14,
    step=1,
    description='Farming Level:',
    style={'description_width': 'initial'},
    disabled=False
)

# Interactive widget for toggling whether to display fertilizers within the graph
fertilToggle = widgets.Checkbox(
    value=True,
    description='Show fertilizers',
    disabled=False
)

# Interactive widget for selecting a growth fertilizer to reference for the dfGrowth dataframe; can be toggled OFF
growthFertilDrop = widgets.Dropdown(
    options=[('', 1), ('Speed-Gro', 0.9), ('Deluxe Speed-Gro', 0.75), ('Hyper Speed-Gro', 0.67)],
    value=1,
    description='Growth Fertilizer',
    style={'description_width': 'initial'},
    disabled=False
)

# Interactive widget for selecting a crop-quality fertilizer to reference for the dfQual dataframe; can be toggled OFF
qualFertilDrop = widgets.Dropdown(
    options=['', 'Basic Fertilizer', 'Quality Fertilizer', 'Deluxe Fertilizer'],
    value='',
    description='Crop-quality Fertilizer',
    style={'description_width': 'initial'},
    disabled=False
)

# Output widget allowing for the addition/removal of both fertilizer dropdowns from the current display
out = widgets.Output()
fertils = widgets.HBox((qualFertilDrop, growthFertilDrop))
graphNav = widgets.HBox((prevPageButton, nextPageButton))

# Behavior

In [ ]:
# Mutable object storing the row entry values to graph the x-axis with, modified by the buttons
xBounds = [0,4]

# Initialize the dataframes to their initial/default states
calc_crop_avgs(dfBase, normalAvgPrices[0], 1)
dfQual = dfBase.copy(deep=True)
dfGrowth = dfBase.copy(deep=True)

# Initialize the visualization to its initial/default state
fig, y_scl = init_graph()
viz = widgets.VBox((fig, graphNav, graphedValueSel, farmingLevelField, fertilToggle, out))

# Declare observer events on the widgets and process the change
graphedValueSel.observe(update_graph, names='value')
fertilToggle.observe(toggle_dropdowns, names='value')
farmingLevelField.observe(process_sell_mult, names='value')
qualFertilDrop.observe(process_sell_mult, names='value')
growthFertilDrop.observe(process_growth_fertil, names='value')
prevPageButton.on_click(partial(modify_selection, xBounds))
nextPageButton.on_click(partial(modify_selection, xBounds))


display(viz)
with out:
    display(fertils)